In [1]:
import ROOT
import numpy as np
import math
from ROOT import TMath
import math
from math import exp
import os

Welcome to JupyROOT 6.18/04


In [2]:
%jsroot on

In [3]:
c_sig=ROOT.TChain("mini")
c_sig.Add("/opendata/H->yy/gamma gamma data/GamGam/Data/data_A.GamGam.root")
c_sig.Add("/opendata/H->yy/gamma gamma data/GamGam/Data/data_B.GamGam.root")
c_sig.Add("/opendata/H->yy/gamma gamma data/GamGam/Data/data_C.GamGam.root")
c_sig.Add("/opendata/H->yy/gamma gamma data/GamGam/Data/data_D.GamGam.root")
c_sig.GetEntries()

7798424

In [4]:
c_ttH=ROOT.TChain("mini")
c_ttH.Add("/opendata/H->yy/gamma gamma data/GamGam/MC/mc_341081.ttH125_gamgam.GamGam.root")
c_ggH=ROOT.TChain("mini")
c_ggH.Add("/opendata/H->yy/gamma gamma data/GamGam/MC/mc_343981.ggH125_gamgam.GamGam.root")
c_VBFH=ROOT.TChain("mini")
c_VBFH.Add("/opendata/H->yy/gamma gamma data/GamGam/MC/mc_345041.VBFH125_gamgam.GamGam.root")
c_WpH=ROOT.TChain("mini")
c_WpH.Add("/opendata/H->yy/gamma gamma data/GamGam/MC/mc_345318.WpH125J_Wincl_gamgam.GamGam.root")
c_ZH=ROOT.TChain("mini")
c_ZH.Add("/opendata/H->yy/gamma gamma data/GamGam/MC/mc_345319.ZH125J_Zincl_gamgam.GamGam.root")
print(c_ttH.GetEntries(), c_ggH.GetEntries(), c_VBFH.GetEntries(), c_WpH.GetEntries(), c_ZH.GetEntries())

576491 1054711 497468 113765 230900


In [5]:
class Analysis:
    def __init__(self, tree, option):
        self.tree = tree
        self.option = option
        self.accumulated_entries = 0
        

    def begin(self):
        nentries = self.tree.GetEntries()
        print("Processing ",nentries," entries in the "+self.option+" chain")
        self.h_yymass = ROOT.TH1F("h_yymass_"+self.option,"Invariant mass of photon pairs",30,105,160)
    
    def process(self):
        for event in self.tree:
            self.accumulated_entries += 1
            
            if (self.tree.trigP):
                nphotons = 0
                p1 = ROOT.TLorentzVector()
                p2 = ROOT.TLorentzVector()
                leading = ROOT.TLorentzVector()
                subleading = ROOT.TLorentzVector()
                diphotonvector = ROOT.TLorentzVector()
                myy = 0

                for iphoton in range(self.tree.photon_n):
                    pt = self.tree.photon_pt[iphoton]
                    eta = self.tree.photon_eta[iphoton]
                    ptcone30 = self.tree.photon_ptcone30[iphoton]
                    etcone20 = self.tree.photon_etcone20[iphoton]
                    
                    if (self.tree.photon_isTightID[iphoton]):
                        if (((pt/1000>25) and (TMath.Abs(eta)<2.37)) and ((TMath.Abs(eta)<1.37) or (TMath.Abs(eta))>1.52)):
                            if ((ptcone30/pt)<.065) and ((etcone20/pt)<.065):
                                nphotons += 1


                if (nphotons == 2):
                    counter = 0
                    for iphoton in range(self.tree.photon_n):
                        pt = self.tree.photon_pt[iphoton]
                        eta = self.tree.photon_eta[iphoton]
                        phi = self.tree.photon_phi[iphoton]
                        ptcone30 = self.tree.photon_ptcone30[iphoton]
                        etcone20 = self.tree.photon_etcone20[iphoton]
                        E = self.tree.photon_E[iphoton]

                        if (self.tree.photon_isTightID[iphoton]):
                            if (((pt/1000>25) and (TMath.Abs(eta)<2.37)) and ((TMath.Abs(eta)<1.37) or (TMath.Abs(eta))>1.52)):
                                if ((ptcone30/pt)<.065) and ((etcone20/pt)<.065):
                                    if counter == 0:
                                        counter += 1
                                        p1.SetPtEtaPhiE (pt, eta, phi, E)
                                    if counter != 0:
                                        p2.SetPtEtaPhiE (pt, eta, phi, E)
                    
                    if p1.Pt()>p2.Pt():
                        leading = p1
                        subleading = p2
                    if p2.Pt()>p1.Pt():
                        leading = p2
                        subleading = p1



                    if (leading.Pt()/1000) > 35 and (subleading.Pt()/1000) > 25:
                        diphotonvector = leading + subleading
                        myy = diphotonvector.M()/1000

                        if (((leading.Pt()/1000)/myy)>.35) and (((subleading.Pt()/1000)/myy)>.25):

                            if myy > 105 and myy < 160:
                                if self.option == "sig":
                                    weight = 1
                                if self.option != "sig":
                                    weight = (self.tree.mcWeight * self.tree.scaleFactor_PILEUP *\
                                              self.tree.scaleFactor_PhotonTRIGGER * self.tree.scaleFactor_PHOTON)
                                self.h_yymass.Fill(myy, weight)


            if (self.accumulated_entries%10000 == 1):
                print("Processed: ",self.accumulated_entries)
    def terminate(self):
        
        output_name = "output/MYPhotonAnalysis_"+self.option+".root"
        print("Saving to file "+output_name)
        file = ROOT.TFile(output_name, "RECREATE")
        self.h_yymass.Write()
        file.Close()
        print("Done saving!")
            
            

In [73]:
a_sig=Analysis(c_sig,"sig")
a_sig.begin()
a_sig.process()
a_sig.terminate()

Processing  7798424  entries in the sig chain
Processed:  1
Processed:  10001
Processed:  20001
Processed:  30001
Processed:  40001
Processed:  50001
Processed:  60001
Processed:  70001
Processed:  80001
Processed:  90001
Processed:  100001
Processed:  110001
Processed:  120001
Processed:  130001
Processed:  140001
Processed:  150001
Processed:  160001
Processed:  170001
Processed:  180001
Processed:  190001
Processed:  200001
Processed:  210001
Processed:  220001
Processed:  230001
Processed:  240001
Processed:  250001
Processed:  260001
Processed:  270001
Processed:  280001
Processed:  290001
Processed:  300001
Processed:  310001
Processed:  320001
Processed:  330001
Processed:  340001
Processed:  350001
Processed:  360001
Processed:  370001
Processed:  380001
Processed:  390001
Processed:  400001
Processed:  410001
Processed:  420001
Processed:  430001
Processed:  440001
Processed:  450001
Processed:  460001
Processed:  470001
Processed:  480001
Processed:  490001
Processed:  500001

Processed:  4140001
Processed:  4150001
Processed:  4160001
Processed:  4170001
Processed:  4180001
Processed:  4190001
Processed:  4200001
Processed:  4210001
Processed:  4220001
Processed:  4230001
Processed:  4240001
Processed:  4250001
Processed:  4260001
Processed:  4270001
Processed:  4280001
Processed:  4290001
Processed:  4300001
Processed:  4310001
Processed:  4320001
Processed:  4330001
Processed:  4340001
Processed:  4350001
Processed:  4360001
Processed:  4370001
Processed:  4380001
Processed:  4390001
Processed:  4400001
Processed:  4410001
Processed:  4420001
Processed:  4430001
Processed:  4440001
Processed:  4450001
Processed:  4460001
Processed:  4470001
Processed:  4480001
Processed:  4490001
Processed:  4500001
Processed:  4510001
Processed:  4520001
Processed:  4530001
Processed:  4540001
Processed:  4550001
Processed:  4560001
Processed:  4570001
Processed:  4580001
Processed:  4590001
Processed:  4600001
Processed:  4610001
Processed:  4620001
Processed:  4630001


In [74]:
a_ttH=Analysis(c_ttH,"ttH")
a_ttH.begin()
a_ttH.process()
a_ttH.terminate()

Processing  576491  entries in the ttH chain
Processed:  1
Processed:  10001
Processed:  20001
Processed:  30001
Processed:  40001
Processed:  50001
Processed:  60001
Processed:  70001
Processed:  80001
Processed:  90001
Processed:  100001
Processed:  110001
Processed:  120001
Processed:  130001
Processed:  140001
Processed:  150001
Processed:  160001
Processed:  170001
Processed:  180001
Processed:  190001
Processed:  200001
Processed:  210001
Processed:  220001
Processed:  230001
Processed:  240001
Processed:  250001
Processed:  260001
Processed:  270001
Processed:  280001
Processed:  290001
Processed:  300001
Processed:  310001
Processed:  320001
Processed:  330001
Processed:  340001
Processed:  350001
Processed:  360001
Processed:  370001
Processed:  380001
Processed:  390001
Processed:  400001
Processed:  410001
Processed:  420001
Processed:  430001
Processed:  440001
Processed:  450001
Processed:  460001
Processed:  470001
Processed:  480001
Processed:  490001
Processed:  500001


In [75]:
a_ggH=Analysis(c_ggH,"ggH")
a_ggH.begin()
a_ggH.process()
a_ggH.terminate()

Processing  1054711  entries in the ggH chain
Processed:  1
Processed:  10001
Processed:  20001
Processed:  30001
Processed:  40001
Processed:  50001
Processed:  60001
Processed:  70001
Processed:  80001
Processed:  90001
Processed:  100001
Processed:  110001
Processed:  120001
Processed:  130001
Processed:  140001
Processed:  150001
Processed:  160001
Processed:  170001
Processed:  180001
Processed:  190001
Processed:  200001
Processed:  210001
Processed:  220001
Processed:  230001
Processed:  240001
Processed:  250001
Processed:  260001
Processed:  270001
Processed:  280001
Processed:  290001
Processed:  300001
Processed:  310001
Processed:  320001
Processed:  330001
Processed:  340001
Processed:  350001
Processed:  360001
Processed:  370001
Processed:  380001
Processed:  390001
Processed:  400001
Processed:  410001
Processed:  420001
Processed:  430001
Processed:  440001
Processed:  450001
Processed:  460001
Processed:  470001
Processed:  480001
Processed:  490001
Processed:  500001

In [76]:
a_VBFH=Analysis(c_VBFH,"VBFH")
a_VBFH.begin()
a_VBFH.process()
a_VBFH.terminate()

Processing  497468  entries in the VBFH chain
Processed:  1
Processed:  10001
Processed:  20001
Processed:  30001
Processed:  40001
Processed:  50001
Processed:  60001
Processed:  70001
Processed:  80001
Processed:  90001
Processed:  100001
Processed:  110001
Processed:  120001
Processed:  130001
Processed:  140001
Processed:  150001
Processed:  160001
Processed:  170001
Processed:  180001
Processed:  190001
Processed:  200001
Processed:  210001
Processed:  220001
Processed:  230001
Processed:  240001
Processed:  250001
Processed:  260001
Processed:  270001
Processed:  280001
Processed:  290001
Processed:  300001
Processed:  310001
Processed:  320001
Processed:  330001
Processed:  340001
Processed:  350001
Processed:  360001
Processed:  370001
Processed:  380001
Processed:  390001
Processed:  400001
Processed:  410001
Processed:  420001
Processed:  430001
Processed:  440001
Processed:  450001
Processed:  460001
Processed:  470001
Processed:  480001
Processed:  490001
Saving to file out

In [77]:
a_WpH=Analysis(c_WpH,"WpH")
a_WpH.begin()
a_WpH.process()
a_WpH.terminate()

Processing  113765  entries in the WpH chain
Processed:  1
Processed:  10001
Processed:  20001
Processed:  30001
Processed:  40001
Processed:  50001
Processed:  60001
Processed:  70001
Processed:  80001
Processed:  90001
Processed:  100001
Processed:  110001
Saving to file output/MYPhotonAnalysis_WpH.root
Done saving!


In [78]:
a_ZH=Analysis(c_ZH,"ZH")
a_ZH.begin()
a_ZH.process()
a_ZH.terminate()

Processing  230900  entries in the ZH chain
Processed:  1
Processed:  10001
Processed:  20001
Processed:  30001
Processed:  40001
Processed:  50001
Processed:  60001
Processed:  70001
Processed:  80001
Processed:  90001
Processed:  100001
Processed:  110001
Processed:  120001
Processed:  130001
Processed:  140001
Processed:  150001
Processed:  160001
Processed:  170001
Processed:  180001
Processed:  190001
Processed:  200001
Processed:  210001
Processed:  220001
Processed:  230001
Saving to file output/MYPhotonAnalysis_ZH.root
Done saving!


In [6]:
canvases = {}
files = {}
histos = {}
legends={}
canvases2 = {}
files2 = {}
histos2 = {}
legends2={}
#
def Plotter1():
    # Tags for plots and samples
    plots = ["yymass"]
    samples = ["sig","ggH","ttH","VBFH","WpH","ZH"]
    
    #scaling and colors
    lum = 10064
    scalefactors = {"sig":1,"ttH":(lum*.0000026433864/55922617.6297),"ggH":(lum*.102/55922617.6297),\
                    "VBFH":(lum*.008518764/3441426.13711),"WpH":(lum*.0019654512/213799.958463),"ZH":(lum*.0017347836/358401.082034)}
    fillColors = {"sig":2, "ttH":4, "ggH":4, "VBFH":4, "WpH":4, "ZH":4}
    fillStyles = {"sig":3003, "ttH":3003, "ggH":3003, "VBFH":3003, "WpH":3003, "ZH":3003}
    xLabels = {"yymass":"diphoton invariant mass"}
    yLabels = {"yymass":"Number/bin"}
    legendLabels = {"sig":"Signal (H #rightarrow yy)", "ttH":"ttH", "ggH":"ggH", "VBFH":"VBFH", "WpH":"WpH", "ZH":"ZH"}
    # Read in histograms, manipulate settings
    for tag in samples:
        filename = "output/MYPhotonAnalysis_"+tag+".root"
        print("Opening "+filename)
        files[tag] = ROOT.TFile(filename,"READ")
        for h in plots:
            legends[h+"_"+tag] = ROOT.TLegend(0.5, 0.7, 0.9, 0.9)
            histo=files[tag].Get("h_"+h+"_"+tag)
            histo.SetFillColor(fillColors[tag])
            histo.SetLineColor(fillColors[tag])
            histo.SetFillStyle(fillStyles[tag])
            histo.Scale(scalefactors[tag])
            histo.GetXaxis().SetTitle(xLabels[h])
            histo.GetYaxis().SetTitle(yLabels[h])
            histo.SetStats(0)
            histos[h+"_"+tag] = histo
            canvases[h+"_"+tag] = ROOT.TCanvas(h+"_"+tag,"Canvas for "+h+"_"+tag, 800, 600)
            legends[h+"_"+tag].AddEntry(histos[h+"_"+tag],legendLabels[tag],"f")
            histos[h+"_"+tag].Draw()
            legends[h+"_"+tag].Draw()
            
            

    #overlays
    for h in plots:
        legends2[h] = ROOT.TLegend(0.5, 0.7, 0.9, 0.9)
        canvases2[h] = ROOT.TCanvas(h,"Canvas for "+h, 800, 600)
        for tag in samples:
            if tag != "sig" and tag != "ggH":
                #legends2[h].AddEntry(histos[h+"_"+tag],legendLabels[tag],"f")
                histos[h+"_ggH"].Add(histos[h+"_"+tag])
        
        legends2[h].AddEntry("Sum of hhG+ttH+VBFH+WpH+ZH")
        legends2[h].Draw()
        histos[h+"_ggH"].SetFillStyle(2)
        histos[h+"_ggH"].SetLineStyle(1)
        histos[h+"_ggH"].SetLineWidth(2)
        histos[h+"_ggH"].SetLineColor(ROOT.kBlack)
        histos[h+"_ggH"].Draw()
        legends2[h].Draw()
        


In [7]:
Plotter1()

Opening output/MYPhotonAnalysis_sig.root
Opening output/MYPhotonAnalysis_ggH.root
Opening output/MYPhotonAnalysis_ttH.root
Opening output/MYPhotonAnalysis_VBFH.root
Opening output/MYPhotonAnalysis_WpH.root
Opening output/MYPhotonAnalysis_ZH.root


In [8]:
sig2 = histos["yymass_sig"]

In [9]:
polyfit2 = ROOT.TF1("polyfit2", "([0] + [1]*x + [2]*x**2 + [3]*x**3 + [4]*exp((-.5*((x-[5])/[6])**2)))", 105, 160)
polyfit2.SetParameter(5, 143)
polyfit2.SetParLimits(5, 140, 150)
polyfit2.SetParameter(6, 2.39)
polyfit2.SetParLimits(6, 1, 3)
polyfit2.SetLineColor(2)
polyfit2.SetLineStyle(1)
polyfit2.SetLineWidth(2)
sig2.Fit("polyfit2", "S", "E SAME", 105, 160)

datagauss2 = ROOT.TF1("datagauss2", "89.8905*exp((-.5*((x-143.025)/1.81455)**2))", 105, 160)


 FCN=22.6839 FROM HESSE     STATUS=NOT POSDEF     50 CALLS        1536 TOTAL
                     EDM=5.56317e-07    STRATEGY= 1      ERR MATRIX NOT POS-DEF
  EXT PARAMETER                APPROXIMATE        STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           7.79707e+04   6.95268e+01   1.85897e-02  -2.02229e-05
   2  p1          -1.40286e+03   7.46906e-01   3.34467e-04  -1.75469e-03
   3  p2           8.73676e+00   5.14449e-03   2.08300e-06  -4.67381e-01
   4  p3          -1.86027e-02   2.57151e-05   4.43523e-09  -4.49002e+01
   5  p4           8.98905e+01   3.61654e+01   1.47555e-02  -2.47388e-06
   6  p5           1.43025e+02   9.13021e-01   9.12468e-05   5.77910e-04
   7  p6           1.81455e+00   6.00962e-01   2.64515e-04  -1.26571e-04


In [10]:
ROOT.gROOT.SetStyle("ATLAS")
fhisto = ROOT.TCanvas("fhisto", "", 700, 750)
mainplot = ROOT.TPad("mainplot","",0,0.35,1,1)
ratioplot = ROOT.TPad("ratioplot","", 0, 0, 1, .35)
mainplot.SetTickx(False)
mainplot.SetTicky(False)
mainplot.SetLeftMargin(.14)
mainplot.SetRightMargin(.05)
mainplot.SetBottomMargin(0)
ratioplot.SetTickx(False)
ratioplot.SetTicky(False)
ratioplot.SetLeftMargin(.14)
ratioplot.SetRightMargin(.05)
ratioplot.SetTopMargin(0)
ratioplot.SetBottomMargin(0.3)
mainplot.Draw()
ratioplot.Draw()
fhisto.Draw()




mainplot.cd()

histos["yymass_sig"].SetMarkerStyle(20)
histos["yymass_sig"].SetMarkerSize(.5)
histos["yymass_sig"].SetLineWidth(1)
histos["yymass_sig"].SetLineColor(ROOT.kBlack)
histos["yymass_sig"].SetMinimum(1e-3)
histos["yymass_sig"].SetMaximum(6e3)
histos["yymass_sig"].GetYaxis().SetLabelSize(0.045)
histos["yymass_sig"].GetYaxis().SetTitleSize(0.05)
histos["yymass_sig"].SetStats(0)
histos["yymass_sig"].SetTitle("Higgs peak")


polygaus = ROOT.TF1("polygaus", "([0] + [1]*x + [2]*x**2 + [3]*x**3 + [4]*exp((-.5*((x-[5])/[6])**2)))", 105, 160)
polygaus.SetParameter(4, 119.1)
polygaus.SetParLimits(4, 110, 1000)
polygaus.SetParameter(5, 125)
polygaus.SetParLimits(5, 120, 130)
polygaus.SetParameter(6, 2.39)
polygaus.SetParLimits(6, 2, 2.5)
polygaus.SetLineColor(ROOT.kRed)
polygaus.SetLineStyle(1)
polygaus.SetLineWidth(2)
histos["yymass_sig"].Fit("polygaus", "", "E SAME", 105, 160)
polygaus.Draw("SAME")


poly = ROOT.TF1("poly", "([0] + [1]*x + [2]*x**2 + [3]*x**3)", 105, 160)
for i in range (0,3):
    poly.FixParameter(i, polygaus.GetParameter(i))
poly.SetLineColor(4)
poly.SetLineStyle(2)
poly.SetLineWidth(2)
histos["yymass_sig"].Fit("poly", "", "E SAME", 105, 160)
poly.Draw("SAME")

histos["yymass_ggH"].SetLineColor(ROOT.kBlack)
histos["yymass_ggH"].SetLineWidth(2)
histos["yymass_ggH"].SetLineStyle(1)
histos["yymass_ggH"].Draw("SAME")


legend = ROOT.TLegend(0.7, 0.65, .9, .9)
legend.SetTextFont(42)
legend.SetBorderSize(0)
legend.SetTextSize(0.1)
legend.AddEntry(histos["yymass_sig"], "Data" ,"lep")
legend.AddEntry(polygaus, "Signal + Bkg.", "l")
legend.AddEntry(poly, "Background", "l")
legend.AddEntry(histos["yymass_ggH"], "Signal", "l")
legend.Draw("SAME")


ratioplot.cd()

ratio = ROOT.TH1F("ratio","",10000,105,160)
ratio.SetMinimum(-125)
ratio.SetMaximum(250)
ratio.GetXaxis().SetLabelSize(0.08)
ratio.GetXaxis().SetTitleSize(0.12)
ratio.GetXaxis().SetTitleOffset(1.0)
ratio.GetYaxis().SetLabelSize(0.07)
ratio.GetYaxis().SetTitleSize(0.09)
ratio.GetYaxis().SetTitle("Data - Bkg.")
ratio.GetYaxis().CenterTitle()
ratio.GetYaxis().SetTitleOffset(0.7)
ratio.GetYaxis().SetNdivisions(503, False)
ratio.GetYaxis().ChangeLabel(-1, -1, 0)
ratio.GetXaxis().SetTitle("m_{#gamma#gamma} [GeV]")
ratio.Draw()


zero = ROOT.TF1("zero", "0", 105, 160)
zero.SetLineStyle(2)
zero.SetLineWidth(1)
zero.SetLineColor(4)
zero.Draw("SAME")

datagauss = ROOT.TF1("datagauss", "136.626*exp((-.5*((x-124.759)/2.27372)**2))+89.8905*exp((-.5*((x-143.025)/1.81455)**2))", 105, 160)
datagauss.SetLineStyle(1)
datagauss.SetLineWidth(2)
datagauss.SetLineColor(2)
datagauss.Draw("SAME")
#ratiosig = ROOT.TH1F("ratiosig", "ratiosig", 10000, 105, 160)
#ratiosig.Eval(polygaus)
#ratiosig.SetLineColor(2)
#ratiosig.SetLineStyle(1)
#ratiosig.SetLineWidth(2)
#ratiosig.Add(poly, -1)
#ratiosig.Draw("SAME")

#datagauss2 = ROOT.TF1("datagauss2", "89.8905*exp((-.5*((x-143.025)/1.81455)**2))", 105, 160)
#datagauss2.SetLineStyle(1)
#datagauss2.SetLineWidth(2)
#datagauss2.SetLineColor(2)
#datagauss2.Draw("SAME")

dataerror = histos["yymass_sig"].Clone()
dataerror.Add(poly, -1)
for i in range(1, histos["yymass_sig"].GetNbinsX()):
    dataerror.SetBinError(i, histos["yymass_sig"].GetBinError(i))
dataerror.Draw("E SAME")




 FCN=19.4452 FROM HESSE     STATUS=NOT POSDEF     50 CALLS        1421 TOTAL
                     EDM=3.12364e-09    STRATEGY= 1      ERR MATRIX NOT POS-DEF
  EXT PARAMETER                APPROXIMATE        STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           9.53141e+04   7.13849e+01   2.27246e-02   1.85628e-06
   2  p1          -1.79938e+03   7.58463e-01   4.29007e-04   2.40659e-04
   3  p2           1.17250e+01   5.21627e-03   2.79546e-06   3.11105e-02
   4  p3          -2.60316e-02   2.64039e-05   6.20643e-09   4.01278e+00
   5  p4           1.36626e+02   4.39406e+01   1.05455e-04   3.14528e-04
   6  p5           1.24759e+02   8.60146e-01   7.48294e-05   2.72858e-05
   7  p6           2.27372e+00   2.65857e-01   1.11861e-03   1.94083e-05
 FCN=33.0362 FROM MIGRAD    STATUS=CONVERGED      16 CALLS          17 TOTAL
                     EDM=7.1908e-21    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                 

In [19]:
gauss = ROOT.TF1("gaus", "([0]*exp((-.5*((x-[1])/[2])**2)))", 105, 160)
gauss.SetParameter(0, 119.1)
gauss.SetParLimits(0, 110, 130)
gauss.SetParameter(1, 125)
gauss.SetParLimits(1, 120, 130)
gauss.SetParameter(2, 2.39)
gauss.SetParLimits(2, 2, 2.5)
histos["yymass_ggH"].Fit("gaus")




HiggseventsvsMC = ROOT.TCanvas("Higgs events","Higgs events", 600, 600)


Higgs = ROOT.TH1F("Higgs", "Higgs events", 1000, 105, 160)
#Higgs.Eval(polygaus)
#Higgs.SetLineColor(2)
#Higgs.SetLineStyle(1)
#Higgs.SetLineWidth(2)
#Higgs.Add(poly, -1)
Higgs.SetMinimum(-125)
Higgs.SetMaximum(250)
Higgs.Draw()

datagauss5 = ROOT.TF1("datagauss5", "136.626*exp((-.5*((x-124.759)/2.27372)**2))", 105, 160)
datagauss5.SetLineStyle(1)
datagauss5.SetLineWidth(2)
datagauss5.SetLineColor(2)
datagauss5.Draw("SAME")

gauss.SetLineStyle(2)
gauss.SetLineColor(1)
gauss.SetLineWidth(3)
gauss.Draw("SAME")

dataerror2 = histos["yymass_sig"].Clone()
dataerror2.Add(poly, -1)
for i in range(1, histos["yymass_sig"].GetNbinsX()):
    dataerror2.SetBinError(i, histos["yymass_sig"].GetBinError(i))
dataerror2.Draw("E SAME")

legendMCvsdata = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)
legendMCvsdata.SetTextFont(42)
legendMCvsdata.SetBorderSize(0)
legendMCvsdata.SetTextSize(0.7)
legendMCvsdata.AddEntry(gauss, "MC" ,"l")
legendMCvsdata.AddEntry(datagauss5, "Data", "l")

HiggseventsvsMC.Draw()
legendMCvsdata.Draw()

 FCN=9335.64 FROM MIGRAD    STATUS=CONVERGED     108 CALLS         109 TOTAL
                     EDM=3.75941e-08    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.14998e+02   1.35573e-01   7.38704e-04  -6.85676e-03
   2  p1           1.24940e+02   2.95853e-03   2.79191e-05   3.62730e-01
   3  p2           2.50000e+00   3.42412e-04   2.46936e-03** at limit **


Warning in <TCanvas::Constructor>: Deleting canvas with same name: Higgs events
Warning in <TFile::Append>: Replacing existing TH1: Higgs (Potential memory leak).


In [12]:
Higgsgaussfit1 = ROOT.TF1("Higgsgaussfit1", "[0]*exp((-.5*((x-[1])/[2])**2))", 105, 160)
Higgsgaussfit1.SetParameter(0, 119.1)
Higgsgaussfit1.SetParLimits(0, 110, 1000)
Higgsgaussfit1.SetParameter(1, 125)
Higgsgaussfit1.SetParLimits(1, 120, 130)
Higgsgaussfit1.SetParameter(2, 2.39)
Higgsgaussfit1.SetParLimits(2, 2, 2.5)
Higgsevents1 = histos["yymass_sig"].Clone()
Higgsevents1.Add(poly, -1)
Higgsevents1.Fit("Higgsgaussfit1", "S", "E SAME", 115, 135)

polychifit = ROOT.TF1("polychifit", "([0] + [1]*x + [2]*x**2 + [3]*x**3)", 105, 160)
Higgsevents2 = histos["yymass_sig"].Clone()
Higgsevents2.Fit("polychifit", "S", "E SAME", 105, 160)


Higgsevents3 = histos["yymass_sig"].Clone()
Higgsevents3.Add(poly, -1)
Higgsnullfit = ROOT.TF1("Higgsnullfit", "[0]", 105, 160)
Higgsnullfit.SetParameter(0, 0)
Higgsnullfit.SetParLimits(0, -.1, .1)
Higgsevents3.Fit("Higgsnullfit", "S", "E SAME", 115, 135)


Higgsgaussfit4 = ROOT.TF1("Higgsgaussfit4", "[0]*exp((-.5*((x-[1])/[2])**2))", 105, 160)
Higgsgaussfit4.SetParameter(1, 143)
Higgsgaussfit4.SetParLimits(1, 140, 150)
Higgsgaussfit4.SetParameter(2, 2.39)
Higgsgaussfit4.SetParLimits(2, 0, 5)
Higgsevents4 = histos["yymass_sig"].Clone()
Higgsevents4.Add(poly, -1)
Higgsevents4.Fit("Higgsgaussfit4", "S", "E SAME", 132, 152)



test1 = ROOT.TCanvas("test1","MC vs data", 600, 600)
Higgsevents1.Draw()
Higgsgaussfit4.Draw()

test1.Draw()


Higgspeakevents = (Higgsgaussfit1.Integral(115,135) / (55/30))
Secondpeakevents = (Higgsgaussfit4.Integral(132,152)/(55/30))



print("Total events above background in Higgs peak:")
print(Higgspeakevents)
print("Total events above background in 2nd peak peak:")
print(Secondpeakevents)
print("chi^2 for Higgs peak fit")
print(Higgsgaussfit1.GetChisquare())
print("NDF for Higgs peak fit")
print(Higgsgaussfit1.GetNDF())
print("chi^2 for poly fit")
print(polychifit.GetChisquare())
print("NDF for poly fit")
print(polychifit.GetNDF())
print("chi^2 for best fit, polygaus")
print(polygaus.GetChisquare())
print("NDF for best fit, polygaus")
print(polygaus.GetNDF())
print("chi^2 for null hypothesis")
print(Higgsnullfit.GetChisquare())
print("NDF for null hypothesis")
print(Higgsnullfit.GetNDF())
print("chi^2 for 2nd peak gauss fit")
print(Higgsgaussfit4.GetChisquare())
print("NDF 2nd peak gauss fit")
print(Higgsgaussfit4.GetNDF())

Total events above background in Higgs peak:
397.80833244386747
Total events above background in 2nd peak peak:
114.96381875453137
chi^2 for Higgs peak fit
1.9200092269272804
NDF for Higgs peak fit
8
chi^2 for poly fit
27.824734164811165
NDF for poly fit
26
chi^2 for best fit, polygaus
19.44521081301043
NDF for best fit, polygaus
23
chi^2 for null hypothesis
14.876847429578476
NDF for null hypothesis
10
chi^2 for 2nd peak gauss fit
11.44336746523683
NDF 2nd peak gauss fit
8
 FCN=1.92001 FROM MIGRAD    STATUS=CONVERGED      65 CALLS          66 TOTAL
                     EDM=9.91981e-10    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  p0           1.33188e+02   4.34244e+01   2.17697e-04   9.83205e-05
   2  p1           1.24719e+02   8.73456e-01   1.42375e-04   1.48004e-04
   3  p2           2.18456e+00   3.17433e-01   2.17731e-03  -5.36492e-06
 

In [ ]:
HiggseventsvsMC = ROOT.TCanvas("Higgs events","Higgs events", 600, 600)
HiggseventsvsMC2 = ROOT.TH1F("HiggseventsvsMC","Higgs events Data vs MC",30,105,160)
HiggseventsvsMC2.SetMinimum(-125)
HiggseventsvsMC2.SetMaximum(250)
HiggseventsvsMC2.Draw()

#Higgs = ROOT.TH1F("Higgs", "Higgs events", 1000, 105, 160)
#Higgs.Eval(polygaus)
#Higgs.SetLineColor(2)
#Higgs.SetLineStyle(1)
#Higgs.SetLineWidth(2)
#Higgs.Add(poly, -1)
#Higgs.SetMinimum(-125)
#Higgs.SetMaximum(250)
#Higgs.Draw()

gauss = ROOT.TF1("gauss", "([0]*exp((-.5*((x-[1])/[2])**2)))", 105, 160)
gauss.SetParameter(0, 119.1)
gauss.SetParLimits(0, 110, 130)
gauss.SetParameter(1, 125)
gauss.SetParLimits(1, 120, 130)
gauss.SetParameter(2, 2.39)
gauss.SetParLimits(2, 2, 2.5)
histos["yymass_ggH"].Fit("gaus")

gauss.SetMinimum(-125)
gauss.SetMaximum(250)
gauss.SetLineStyle(2)
gauss.SetLineColor(1)
gauss.SetLineWidth(3)
gauss.Draw()

datagauss5 = ROOT.TF1("datagauss5", "136.626*exp((-.5*((x-124.759)/2.27372)**2))", 105, 160)
datagauss5.SetLineStyle(1)
datagauss5.SetLineWidth(2)
datagauss5.SetLineColor(2)
datagauss5.Draw("SAME")


dataerror2 = histos["yymass_sig"].Clone()
dataerror2.Add(poly, -1)
for i in range(1, histos["yymass_sig"].GetNbinsX()):
    dataerror2.SetBinError(i, histos["yymass_sig"].GetBinError(i))
dataerror2.Draw("E SAME")

legendMCvsdata = ROOT.TLegend(0.7, 0.7, 0.9, 0.9)
legendMCvsdata.SetTextFont(42)
legendMCvsdata.SetBorderSize(0)
legendMCvsdata.SetTextSize(0.7)
legendMCvsdata.AddEntry(gauss, "MC" ,"l")
legendMCvsdata.AddEntry(datagauss5, "data", "l")

HiggseventsvsMC.Draw()
legendMCvsdata.Draw()